# Setup


In [1]:
# Import packages
import cv2
import matplotlib.pyplot as plt
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Read images
image1 = cv2.imread('Images/Image6.jpg') # Russian Taxi at Angle and Some Distance
image2 = cv2.imread('Images/Image7.jpg') # Two Russian Plates with Telephoto Lens
image3 = cv2.imread('Images/Image4.jpg') # Ontario Plate
source = list([image1, image2, image3])
# https://www.geeksforgeeks.org/how-to-fix-valueerror-setting-an-array-element-with-a-sequence/
source = np.array(source, dtype = list)

# Define placeholders
gray = np.array([0, 1, 2], dtype = list)
plates = np.array(list(range(3)), dtype = list)
plateszoom = np.array(list(range(3)), dtype = list)

# Convert to Grayscale
for i in range(len(source)):
    gray[i] = cv2.cvtColor(source[i], cv2.COLOR_BGR2GRAY)

# # Load Classifier
rusPlateReader = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_license_plate_rus_16stages.xml')
rusPlateFinder = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')

# Detect Plates

In [4]:
# Detect Plates
# https://github.com/kennethleungty/Car-Plate-Detection-OpenCV-TesseractOCR/blob/main/Car%20Plate%20Detection%20with%20OpenCV%20and%20TesseractOCR.ipynb
def CarplateDetect(image):
    # Color rectangle overlay on grayscale image plate detection area
    carplate_overlay = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)
    carplate_rects = rusPlateFinder.detectMultiScale(carplate_overlay,
                                                     scaleFactor = 1.1,
                                                     minNeighbors = 5) 
    # carplate_img = 0

    for x, y, w, h in carplate_rects:
        cv2.rectangle(carplate_overlay,
                      (x, y),
                      (x + w, y + h),
                      (255, 0, 0),
                      25) 
        
    # for x, y, w, h in carplate_rects:
    #     carplate_img = carplate_overlay[y + 15: y + h - 10,
    #                          x + 15: x + w - 20] 


    return carplate_overlay

In [26]:
carplate_overlay = image2.copy()
carplate_rects = rusPlateFinder.detectMultiScale(carplate_overlay,
                                                    scaleFactor = 1.1,
                                                    minNeighbors = 5) 

for x, y, w, h in carplate_rects:
    cv2.rectangle(carplate_overlay,
                    (x, y),
                    (x + w, y + h),
                    (255, 0, 0),
                    25)

# Zoom on Plates

## Works

In [9]:
def PlateZoom(image):
    carplate_overlay = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)
    carplate_rects = rusPlateFinder.detectMultiScale(carplate_overlay,
                                                     scaleFactor = 1.1,
                                                     minNeighbors = 5)
    for x, y, w, h in carplate_rects:
        carplate_img = carplate_overlay[y + 15: y + h - 10,
                             x + 15: x + w - 20]
    return carplate_img

In [6]:
def carplate_extract(image):
    
    carplate_rects = rusPlateFinder.detectMultiScale(image,scaleFactor=1.1, minNeighbors=5) 

    plates_x = min(carplate_rects[:, 0])
    plates_y = min(carplate_rects[:, 1])
    plates_h = max(carplate_rects[:, 3])
    plates_w = max(carplate_rects[:, 2])
    carplate_img = image[plates_y : plates_y + plates_h, plates_x : plates_x + plates_w]

    return carplate_img

## Experimenting

In [40]:
carplate_rects = rusPlateFinder.detectMultiScale(gray[1],
                                                     scaleFactor = 1.1,
                                                     minNeighbors = 5)

In [42]:
carplate_rects

array([[881, 577, 265,  89],
       [218, 531, 190,  63]], dtype=int32)

In [53]:
for i in range(0, len(carplate_rects)):
    print(i)

0
1


In [47]:
range(0, len(carplate_rects))

range(0, 2)

In [46]:
carplate_rects_stage = carplate_rects[0]

In [54]:
for x, y, w, h in carplate_rects_stage:
    print(x, y, w, h)

TypeError: cannot unpack non-iterable numpy.int32 object

In [60]:
type(carplate_rects_stage[0])


numpy.int32

In [66]:
def PlateZoom(image):
    carplate_overlay = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)
    carplate_rects = rusPlateFinder.detectMultiScale(carplate_overlay,
                                                     scaleFactor = 1.1,
                                                     minNeighbors = 5)
    if len(carplate_rects) < 2:
        for x, y, w, h in carplate_rects:
            carplate_img = carplate_overlay[y + 15: y + h - 10,
                                x + 15: x + w - 20]
    else:
        for i in range(0, len(carplate_rects)):
            carplate_rects_stage = carplate_rects[i]
            carplate_img[i] = carplate_overlay[int(carplate_rects_stage[0]) + 15: int(carplate_rects_stage[1]) + int(carplate_rects_stage[3]) - 10,
                                int(carplate_rects_stage[0]) + 15: int(carplate_rects_stage[0]) + int(carplate_rects_stage[2]) - 20]
        exit
    return carplate_img

In [67]:
plateszoom = PlateZoom(gray[0])

In [32]:
carplate_rects

array([[881, 577, 265,  89],
       [218, 531, 190,  63]], dtype=int32)

In [12]:
cv2.imshow("test", carplate_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

# Plot Results

In [7]:
plates = CarplateDetect(gray[0])
plateszoom = carplate_extract(gray[0])

In [25]:
carplate_rects = rusPlateFinder.detectMultiScale(gray[1],scaleFactor=1.1, minNeighbors=5) 

In [13]:
# cv2.imshow("test", plates)
cv2.imshow("test", plateszoom)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [58]:
for i in range(len(source)):
    plates[i] = CarplateDetect(gray[i])
    plateszoom[i] = carplate_extract(gray[i])

TypeError: tuple indices must be integers or slices, not tuple

In [5]:
for i in range(len(source)):
    plates[i] = CarplateDetect(gray[i])
    # plateszoom[i] = carplate_extract(gray[i])

images = [plates[0], plates[1], plates[2],
          plateszoom[0], plateszoom[1], plateszoom[2]]

In [ ]:
for i in range(3):
    plt.subplot(3, 2, i + 1), plt.imshow(plates[i], 'gray')
    plt.xticks([]), plt.yticks([])
    # plt.title(titles[i])
    plt.tight_layout()
# plt.suptitle('Edge Detection Results', fontsize = 25).set_color('#171819')
plt.tight_layout()
plt.show()